In [30]:
%load_ext autoreload
import pandas as pd
from tqdm.notebook import tqdm
import csv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Ідеї


1. Не враховувати останній абзац. В багатьох це може бути назва каналу, в якій є місто / район етс
1. Можливо натренувати окрему модель, яка детектить чи це назва каналу чи ні (приблизно зроузміло, як зібрати датасет)
1. Юзати labeling sample як валідацію 
1. 

## Baseline

In [2]:
val_df = pd.read_csv("data/mantis_analytics_location_detection/labeling_sample.csv", converters={"locations": eval})
test_df = pd.read_csv("data/mantis_analytics_location_detection/test.csv", converters={"locations": eval})

In [3]:
val_df

,text,locations,loc_markers
0,-261.310,[],[]
1,"Проспект Бандери стоїть, як і пів міста… довго...",[Проспект Бандери],"[(0, 16)]"
2,☢️Радіаційний фон (мкР/год) по Київщині станом...,"[Київщині, Чорнобиль, Київ, Тетерів, Фастів, Б...","[(31, 39), (68, 77), (83, 87), (93, 100), (105..."
3,"Друзі. Останні дві ночі, і ще декілька наступн...",[проспекті Івасюка],"[(134, 151)]"
4,Сьогодні з Києва до Хелма вперше вирушив новий...,"[Києва, Хелма]","[(11, 16), (20, 25)]"
5,Подовження роботи закладів харчування\n\nВладо...,[Києву],"[(861, 866)]"
6,"Нарешті офіційні новини, які можна розповідати...",[],[]
7,ДТП біля заїзду в ТЦ Аркадія\n\n📍Індустріальни...,[ТЦ Аркадія],"[(18, 28)]"
8,“Київський метрополітен” отримав ще 6 вагонів ...,[],[]
9,В результате массированной атаки врага ночью в...,"[Фастовском, Бучанском]","[(47, 57), (60, 69)]"


In [4]:
%autoreload 2
from get_predictions import get_word_predictions

In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained('ukr-models/uk-ner')
model = AutoModelForTokenClassification.from_pretrained('ukr-models/uk-ner')

In [6]:
words, ners = get_word_predictions(model, tokenizer, val_df['text'])

In [7]:
for ner, word in zip(ners[22], words[22]):
    if ner in ["I-LOC", "B-LOC"]:
        print(word)

Броварах
Київщині
Києві
Подолі


In [12]:
words_test = []
ners_test = []

for i in tqdm(range(0, 471, 10)):
    # print(f"Batch {i}")
    words_test_i, ners_test_i = get_word_predictions(model, tokenizer, test_df.loc[i:i+10, 'text'])
    words_test.extend(words_test_i)
    ners_test.extend(ners_test_i)

  0%|          | 0/48 [00:00<?, ?it/s]

In [24]:
res_col = []
for s_ind in range(len(test_df)):
    res = []
    for i, (ner, word) in enumerate(zip(ners_test[s_ind], words_test[s_ind])):
        # if ner == "I-LOC":
        #     res[-1] = res[-1] + " " + word
        # elif ner == "B-LOC":
        #     res.append(word)
        if ner in ["I-LOC", "B-LOC"]:
            res.append(word)
    res_col.append(res)

In [26]:
test_df['locations'] = res_col

In [34]:
test_df[['text_id', 'locations']].to_csv("uk_ner_baseline.csv", index=False)

## Validation

In [36]:
import re
from typing import List

In [46]:
def hard_processing(text):
    text = re.sub(r"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\b\w\b\s?', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

def comp_metric(y_true: List[List[str]], y_pred: List[List[str]]):
    assert len(y_true) == len(y_pred)
    tp, fp, fn, p,  = 0.0, 0.0, 0.0, 0.0

    for y_true_sample, y_pred_sample in zip(y_true, y_pred):
        y_true_sample = set([hard_processing(s) for s in y_true_sample])
        y_pred_sample = set([hard_processing(s) for s in y_pred_sample])

        tp += len(y_true_sample & y_pred_sample)
        fp += len(y_pred_sample - y_true_sample)
        fn += len(y_true_sample - y_pred_sample)
        p += len(y_true_sample)

    if tp + fp == 0:
        if p == 0:
            precision = 1.0
        else:
            precision = 0.0
    else:
        precision = tp/(tp + fp)

    if tp + fn == 0:
        if p == 0:
            recall = 1.0
        else:
            recall = 0.0
    else:
        recall = tp/(tp + fn)

    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)
    
    # print(precision, recall)
    return round(f1, 8)


In [39]:
res_col = []
for s_ind in range(len(val_df)):
    res = []
    for i, (ner, word) in enumerate(zip(ners[s_ind], words[s_ind])):
        if ner in ["I-LOC", "B-LOC"]:
            res.append(word)
    res_col.append(res)

In [40]:
val_df['locations_pred'] = res_col

In [41]:
val_df

,text,locations,loc_markers,locations_pred
0,-261.310,[],[],[]
1,"Проспект Бандери стоїть, як і пів міста… довго...",[Проспект Бандери],"[(0, 16)]",[]
2,☢️Радіаційний фон (мкР/год) по Київщині станом...,"[Київщині, Чорнобиль, Київ, Тетерів, Фастів, Б...","[(31, 39), (68, 77), (83, 87), (93, 100), (105...","[Київщині, Київ, Бориспіль]"
3,"Друзі. Останні дві ночі, і ще декілька наступн...",[проспекті Івасюка],"[(134, 151)]","[проспекті, Івасюка]"
4,Сьогодні з Києва до Хелма вперше вирушив новий...,"[Києва, Хелма]","[(11, 16), (20, 25)]","[Києва, Хелма]"
5,Подовження роботи закладів харчування\n\nВладо...,[Києву],"[(861, 866)]","[Києва, Києву, Україні]"
6,"Нарешті офіційні новини, які можна розповідати...",[],[],[України]
7,ДТП біля заїзду в ТЦ Аркадія\n\n📍Індустріальни...,[ТЦ Аркадія],"[(18, 28)]","[ТЦ, Аркадія]"
8,“Київський метрополітен” отримав ще 6 вагонів ...,[],[],[]
9,В результате массированной атаки врага ночью в...,"[Фастовском, Бучанском]","[(47, 57), (60, 69)]","[Фастовском, Бучанском]"


In [47]:
comp_metric(val_df['locations'], val_df['locations_pred'])

0.41463415